# Lightpost Demo
Welcome! Here we demo some basic tasks done in Lightpost, a lightweight, automated neural networks training framework written on top of the incredible PyTorch framework. It is made with extensibility and ease of use in mind.

**Contents**
* Basic Classification task
* Text Classification Task
* Deconstructing Lightpost ```Engine```s and ```Datapipe```s
* Using your own PyTorch models with Lightpost

---

## Basic Classification Task
Here we'll test on the iris dataset.

In [1]:
from lightpost.datapipe import Datapipe
from lightpost.estimators import MLPClassifier
from lightpost.engine import Engine

from sklearn.datasets import load_iris
d = load_iris()

We use the basic ```Datapipe``` pipeline from the ```lightpost.datapipe``` API to load our data. It takes in two arrays: an array of features, and an array of targets. Since it's the most basic pipeline, it doesn't perform any preprocessing which other pipelines perform. It will then construct an iterator of randomized batches using the data. We batch it in groups of 32.

We'll also use the basic ```MLPClassifier``` from the ```lightpost.estimators``` API. It's basically a series of fully-connected layers with ReLU activations. We give it the input dimensions and output dimensions of the iris dataset, and give it two hidden layers with 128 neurons each.

Then we'll construct our training engine. We give it our data pipeline, our model, a loss function, and an optimizer.

We can opt to use **Tensorboard**, TensorFlow's visualization tool using the ```use_tensorboard``` option. This will create a logging directory called ```runs``` on the same directory as your working files. To run Tensorboard, run ```tensorboard --logdir runs``` in a command line interface.

In [2]:
pipe = Datapipe(d.data, d.target, batch_size=32)
model = MLPClassifier(input_dim=4, hidden_dim=128, output_dim=3, num_layers=2)
engine = Engine(pipeline=pipe, model=model, criterion='cross_entropy', optimizer='adam', use_tensorboard=True)

We'll train the model using the engine by invoking the ```engine.fit()``` method. We train it for 1000 epochs and print the logs every 100 iterations. We also disable tqdm (the progress bars) so it doesn't overload our window.

In [3]:
engine.fit(epochs=1000, print_every=100, disable_tqdm=True)

Epoch   100 | Train Loss 1.0537 | Train Acc 0.3438 | Val Loss 1.0853 | Val Acc 0.2240
Epoch   200 | Train Loss 0.9970 | Train Acc 0.6562 | Val Loss 1.0201 | Val Acc 0.6458
Epoch   300 | Train Loss 0.9270 | Train Acc 0.6562 | Val Loss 0.9430 | Val Acc 0.5938
Epoch   400 | Train Loss 0.8650 | Train Acc 0.6641 | Val Loss 0.8377 | Val Acc 0.7448
Epoch   500 | Train Loss 0.8255 | Train Acc 0.8672 | Val Loss 0.8062 | Val Acc 0.8073
Epoch   600 | Train Loss 0.7927 | Train Acc 0.9688 | Val Loss 0.7889 | Val Acc 0.9844
Epoch   700 | Train Loss 0.7790 | Train Acc 0.9766 | Val Loss 0.7666 | Val Acc 1.0000
Epoch   800 | Train Loss 0.7652 | Train Acc 0.9375 | Val Loss 0.7482 | Val Acc 0.9844
Epoch   900 | Train Loss 0.7438 | Train Acc 0.9766 | Val Loss 0.7395 | Val Acc 1.0000
Epoch  1000 | Train Loss 0.7331 | Train Acc 0.9609 | Val Loss 0.7324 | Val Acc 0.9844


We can use our model to predict values. Here we'll use the ```engine.predict()``` method to make inferences from the entire iris dataset (which is stored in ```pipe.X```). The first five outputs are shown here.

In [4]:
predictions = engine.predict(pipe.X)
print(predictions.shape)
print(predictions[:5])

torch.Size([150, 3])
tensor([[0.9877, 0.0520, 0.0012],
        [0.9788, 0.0789, 0.0023],
        [0.9828, 0.0651, 0.0020],
        [0.9761, 0.0842, 0.0027],
        [0.9881, 0.0501, 0.0012]], grad_fn=<SliceBackward>)


We can also evaluate the model on a holdout testing set as needed using the ```engine.evaluate()``` method. For illustration, we'll use the entire iris dataset in this example.

In [5]:
loss, acc = engine.evaluate_model(pipe.X, pipe.y)
print('Total Loss: {:.4f} | Total Accuracy: {:.4f}'.format(loss, acc))

Total Loss: 0.7283 | Total Accuracy: 0.6667


---
## Text Classification
For this example, we'll perform a simple text classification task on a subset of the Quora Insincere Comments dataset.

In [6]:
from lightpost.datapipe import Textpipe
from lightpost.estimators import LSTMClassifier
from lightpost.engine import Engine

For NLP tasks, we use a ```Textpipe``` from the ```lightpost.datapipe``` API. This pipeline will automatically do all the preprocessing for the dataset (tokenization, padding, truncation, vocabulary building, etc) as well as building the word embeddings, if one is provided. All corpus information are saved within the pipeline for use later on as needed. Please see the documentation (by typing ```Textpipe?``` in a Jupyter notebook cell) for more details.

In [7]:
pipe = Textpipe(path='quora/train_mini.csv', text='question_text', target='target', maxlen=50, 
                pretrained_embeddings=True, embed_path='quora/wiki.en.vec', embed_dim=300)

100%|██████████| 10000/10000 [00:00<00:00, 92524.95it/s]


We then use an LSTM-based sequence classifier provided by the ```lightpost.estimators``` API. We'll pass it the embedding layer constructed by the pipeline, and give it 128 hidden units in it's one hidden layer. The ```LSTMClassifier``` is very flexible in terms of customization. For this example, we'll make it stack two recurrent, bidirectional layers, each with 0.2 recurrent dropout rate. The fully-connected layer will have a dropout of 0.5.

In [8]:
model = LSTMClassifier(pretrained=pipe.embedding, embedding_dim=pipe.embed_dim, 
                       hidden_dim=128, output_dim=2, bidirectional=True, recur_layers=2, 
                       recur_dropout=0.2, dropout=0.5)

We construct the training engine by passing the pipeline and the model. In this example, we set a learning rate scheduer to decay the learning rate as the validation loss plateaus.

In [9]:
engine = Engine(pipeline=pipe, model=model, criterion='cross_entropy', optimizer='adam', scheduler='plateau')

Training is done using the ```engine.fit()``` method. We train for five epochs and print the logs every 1 iteration. We also enable the progress bars, which will show the progress of each epoch.

In [10]:
engine.fit(epochs=5, print_every=1, disable_tqdm=False)

  0%|          | 0/235 [00:00<?, ?it/s]

Epoch     1 | Train Loss 0.6931 | Train Acc 0.5043 | Val Loss 0.6931 | Val Acc 0.5103


  0%|          | 0/235 [00:00<?, ?it/s]

Epoch     2 | Train Loss 0.6932 | Train Acc 0.5049 | Val Loss 0.6930 | Val Acc 0.5162


  0%|          | 0/235 [00:00<?, ?it/s]

Epoch     3 | Train Loss 0.6720 | Train Acc 0.5627 | Val Loss 0.5991 | Val Acc 0.6962


  0%|          | 0/235 [00:00<?, ?it/s]

Epoch     4 | Train Loss 0.5824 | Train Acc 0.7192 | Val Loss 0.5649 | Val Acc 0.7555


100%|██████████| 79/79 [00:06<00:00, 12.38it/s]

Epoch     5 | Train Loss 0.4926 | Train Acc 0.7805 | Val Loss 0.4798 | Val Acc 0.7860


Likewise, we'll make inferences on some data. For this example, we'll feed the first five sequences of the dataset into the model to get it's log-softmax predictions (the ```LSTMClassifier``` uses log softmax on it's final layer).

In [11]:
predictions = engine.predict(pipe.X[:5])
print(predictions.shape)
print(predictions)

torch.Size([5, 2])
tensor([[-0.1055, -2.3010],
        [-0.5954, -0.8015],
        [-2.0665, -0.1354],
        [-0.7965, -0.5995],
        [-0.5310, -0.8868]], grad_fn=<LogSoftmaxBackward>)


To save the model, we use the ```engine.save_weights()``` method.

In [12]:
engine.save_weights('model.pt')

---
## Deconstruction
As a framework made up of APIs and wrappers, Lightpost's idioms can be deconstructed and used as needed. Here' we can see the contents of a Text Pipeline.

In [13]:
pipe.X

tensor([[ 1872,  4528,  4035,  ...,  3471,  3471,  3471],
        [11941, 11758, 10017,  ...,  3471,  3471,  3471],
        [ 8593,  2863, 15093,  ...,  3471,  3471,  3471],
        ...,
        [ 2076, 15957, 16461,  ...,  3471,  3471,  3471],
        [ 1872,  7954, 14372,  ...,  3471,  3471,  3471],
        [ 2076,  9782,  4226,  ...,  3471,  3471,  3471]])

In [14]:
pipe.y

tensor([0, 1, 1,  ..., 0, 1, 0])

The vocabulary dictionary (and it's reverse) is also stored in the pipeline.

In [15]:
pipe.vocab_dict

{0: 'finding',
 1: 'gentlemen',
 2: 'coordinating',
 3: 'Indonesians',
 4: 'Rights',
 5: 'footage',
 6: 'hefemales',
 7: 'Wildlife',
 8: 'molar',
 9: 'metallurgy',
 10: 'rumours',
 11: 'artistic',
 12: 'substitutes',
 13: 'mentioning',
 14: 'alluring',
 15: 'budget',
 16: 'BC',
 17: 'organs',
 18: 'dreamworld',
 19: 'criminalizing',
 20: 'Asharam',
 21: 'filter',
 22: 'stolen',
 23: 'tonality',
 24: 'solution',
 25: 'Rajput',
 26: 'Beach',
 27: '2005',
 28: 'keep',
 29: 'technically',
 30: 'parenting',
 31: 'involve',
 32: 'Tirupathi',
 33: 'CA',
 34: 'sperm',
 35: 'comfort',
 36: 'spina',
 37: 'States',
 38: 'Gilgit',
 39: 'esteem',
 40: 'limiting',
 41: 'disrespect',
 42: 'hindni',
 43: 'savvy',
 44: 'obligations',
 45: 'now',
 46: 'consciousness',
 47: 'cut',
 48: 'elmo',
 49: 'voluntarily',
 50: 'switched',
 51: 'drafted',
 52: 'head',
 53: 'earliest',
 54: 'harm',
 55: 'Eurocentrics',
 56: 'Vegas',
 57: 'lawfully',
 58: 'benchmark',
 59: 'secularism',
 60: 'GoFundMe',
 61: 'Good',

Likewise, the ```Engine``` objects could also be deconstructed.

Accessing ```engine.model``` returns a PyTorch ```nn.Module```-based model.

In [16]:
engine.model

LSTMClassifier(
  (embedding): Embedding(16970, 300)
  (rnn): LSTM(300, 128, num_layers=2, dropout=0.2, bidirectional=True)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
  (dropout): Dropout(p=0.5)
)

---
## Extensibility
It is easy to use your own PyTorch layers and models in Lightpost.

Let's use the ```lightpost.engine``` API together with our own models.

In [17]:
from lightpost.datapipe import Datapipe
from lightpost.estimators import MLPClassifier
from lightpost.engine import Engine

from sklearn.datasets import load_iris
d = load_iris()

We'll use PyTorch's ```Sequential``` interface to stack some layers together to use as our model. We'll use the iris dataset again so we'll pass in the dimensions of the iris dataset.

In [18]:
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Linear(4, 128),
    nn.ReLU(),
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Linear(128, 3),
    nn.Sigmoid()
)

If we'd like, we can subclass the ```torch.nn.Module``` class to construct a model we can parameterize and use it. The above cell is equivalent with this one:

In [19]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, X):
        out = torch.relu(self.fc1(X))
        out = torch.relu(self.fc2(out))
        out = torch.relu(self.fc3(out))
        out = torch.sigmoid(self.fc4(out))
        return out
    
model = MLP(input_dim=4, hidden_dim=128, output_dim=3)

While more verbose, the above code is more flexible in terms of how tensors are propagated through the layers. We can control forward-propagation this way using control statements as needed, which is good if we'd like to implement complex models. Creating a class for our model is also good for experimenting on multiple instances of the model with different hyperparameters. Combining this with the ```lightpost.engine``` API makes testing multiple models easy. 

In truth, Lightpost was built with this quick prototyping task in mind! Case in point, we'll test it out here.

In [20]:
pipe = Datapipe(d.data, d.target, batch_size=32)
model = MLP(input_dim=4, hidden_dim=128, output_dim=3)

The ```Engine``` interface is also extensible. Here, we'll use our own optimizer and loss functions. For more information on how these are written, consult the PyTorch documentation!

In [21]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9, nesterov=True)
criterion = nn.CrossEntropyLoss()
engine = Engine(pipeline=pipe, model=model, criterion=criterion, optimizer=optimizer)

The ```Engine``` interface essentially becomes a training wrapper. Here we train the model using the engine object we created.

In [22]:
engine.fit(epochs=1000, print_every=100, disable_tqdm=True)

Epoch   100 | Train Loss 1.0783 | Train Acc 0.6641 | Val Loss 1.0923 | Val Acc 0.5104
Epoch   200 | Train Loss 1.0626 | Train Acc 0.6797 | Val Loss 1.0703 | Val Acc 0.5781
Epoch   300 | Train Loss 1.0459 | Train Acc 0.6797 | Val Loss 1.0483 | Val Acc 0.6458
Epoch   400 | Train Loss 1.0247 | Train Acc 0.6875 | Val Loss 1.0337 | Val Acc 0.5781
Epoch   500 | Train Loss 1.0038 | Train Acc 0.6484 | Val Loss 1.0049 | Val Acc 0.6458
Epoch   600 | Train Loss 0.9702 | Train Acc 0.6641 | Val Loss 0.9663 | Val Acc 0.7135
Epoch   700 | Train Loss 0.9271 | Train Acc 0.6875 | Val Loss 0.9400 | Val Acc 0.6458
Epoch   800 | Train Loss 0.8947 | Train Acc 0.6797 | Val Loss 0.9205 | Val Acc 0.5781
Epoch   900 | Train Loss 0.8634 | Train Acc 0.6797 | Val Loss 0.8987 | Val Acc 0.5938
Epoch  1000 | Train Loss 0.8377 | Train Acc 0.7422 | Val Loss 0.8483 | Val Acc 0.8594


---
## Utilities
Lightpost provides many utility functions in the form of the ```lightpost.utils``` interface.

In [23]:
from lightpost.utils.general import split         # Splits a dataset into training and testing sets
from lightpost.utils.text import tokenize_array   # Turns an array of untokenized sequences to an array of token sequences
from lightpost.utils.text import pad              # Pads/Truncates a sequence of tokens according to a max length

Explore the interface to see which you can use!

------------------

*The Lightpost Project (C) 2018 The DLSU Machine Learning Group.*

*Lightpost is developed, maintained, and managed by the DLSU Machine Learning Group as an internal tool for use within the DLSU Center for Complexity and Emerging Technologies laboratory. Please report all bugs and issues encountered in the Lightpost GitHub repo, or contact the developers.*